In [1]:
import pandas as pd
import polars as pl
import numpy as np
import gc
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import StratifiedGroupKFold

In [2]:
class CONFIG:
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.05
    # record num between 30000 and 35000
    start_dt = 950

In [3]:
# Use last 2 parquets
train = pl.scan_parquet(
    f"/root/autodl-tmp/train.parquet"
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).filter(
    pl.col("date_id").gt(CONFIG.start_dt)
)

In [4]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
    )

# the data in the last record represented the closing price for that day.
# I used the last record because I assumed that the next day's value would be 
# closer to the previous day's closing price than the previous day's average value.
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
lags

In [5]:
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")
train

In [6]:
len_train   = train.select(pl.col("date_id")).collect().shape[0]
valid_records = int(len_train * CONFIG.valid_ratio)
len_ofl_mdl = len_train - valid_records
last_tr_dt  = train.select(pl.col("date_id")).collect().row(len_ofl_mdl)[0]

print(f"\n len_train = {len_train}")
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
print(f"\n---> Last offline train date = {last_tr_dt}\n")

training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data   = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 27250168

 len_ofl_mdl = 25887660

---> Last offline train date = 1662



In [7]:
training_data.collect().\
write_parquet(
    f"training.parquet", partition_by = "date_id",
)

In [8]:
validation_data.collect().\
write_parquet(
    "validation.parquet", partition_by = "date_id",
)